# NE 250 – Homework 1
###### 9/22/2017

## Problem 1

The number of molecules, $N$, found in a sample of a compound with mass $M$ is 
$$ N(\cdot) = \frac{M(\cdot) N_A}{m(\cdot)} $$
where $m$ is the molar mass of the compound, and $N_A$ is Avogadro's number ($N_A = 6.022 \times 10^{23}$ molecules per mole). Since each actinide isotope is found only once in a molecule of its respective oxide, $N$ also gives the number of atoms of an isotope in the sample. 

In [1]:
def molecule_count(M,m):
    N_A = 6.022e23
    N = M*N_A/m
    return N

To find $N$, we begin by finding the masses of the compounds in the fuel. We can decompose the total mass of the fuel, $M_f$, into its mixed oxide components:
$$ M_f = M(\text{UO}_2) + M(\text{PuO}_2). $$

Given a weight percent for plutonium, $w_{\text{P}}$, we note that $w_{\text{U}} = 1-w_{\text{P}}$ and so calculate the total masses of both the UO$_2$ and the PuO$_2$ to be
$$ M(\text{UO}_2) = (1-w_{\text{P}})M_f	$$
$$ M(\text{PuO}_2) = w_{\text{P}}M_f. $$

In [2]:
w_p = 0.3

In [3]:
def mass_oxide(w,M_f):
    M_oxide = w*M_f
    return M_oxide

The mass of the oxide component for a given isotope is given by 
$$ M(^{i}\text{XO}_2) = w_i M(\text{XO}_2) $$ 
where $w_i$ is the weight percent of the oxide of isotope $^{i}X$ out of element $X$.

In [4]:
def mass_isotope_oxide(w_i,M_oxide):
    M_i_oxide = w_i*M_oxide
    return M_i_oxide

In [5]:
# Provided weight percents
w = {'U238': 1, # all U is U238
     'Pu239': 0.697,
     'Pu240': 0.218,
     'Pu241': 0.058,
     'Pu242': 0.027
    }

Next, we must determine the molar masses of the various oxides. In general,
$$ m(^{i}\text{XO}_2) = m(^{i}\text{X}) + 2m(\text{O}), $$

In [6]:
def molar_mass_isotope_oxide(m_i,m_o):
    m_i_oxide = m_i + 2*m_o
    return m_i_oxide

Finally, we use both the total mass of a compound with its molar mass in the original formula, using provided or tabulated values:

In [9]:
# Tabulated molar masses [g/mol]
m = {'U238':  238.051,
     'Pu239': 239.052,
     'Pu240': 240.054,
     'Pu241': 241.057,
     'Pu242': 242.059,
     'O16': 15.995
    }

In [10]:
# Assume 1 gram of total fuel
M_f = 1

isotopes = ['U238','Pu239','Pu240','Pu241','Pu242']
for i in isotopes:
    M_i_ox = mass_isotope_oxide(w[i],mass_oxide(w_p,M_f))
    m_i_ox = molar_mass_isotope_oaxide(m[i],m['O16'])
    N = molecule_count(M_i_ox,m_i_ox)
    print(i,': ',N)

U238 :  6.690095207764747e+20
Pu239 :  4.645775193512445e+20
Pu240 :  1.4477025775242242e+20
Pu241 :  3.837537127307754e+19
Pu242 :  1.7799079726618233e+19


## Problem 2

The mean free path of a particle is given by the formula
$$ \lambda = \frac{1}{\Sigma_t} $$
when scattering is considered to be isotropic.